In [9]:
import os
import pandas as pd
import json
import re
import zipfile

# Steam basic info dataset
- https://www.kaggle.com/najzeko/steam-reviews-2021  
- This dataset only contains games up to 2019, but contains more detailed info such as genre, number of players, etc.
- Use info from this dataset to extract useful info from more updated dataset

In [32]:
base_dir = os.path.dirname(os.path.dirname((os.path.dirname(os.getcwd()))))
data_dir = os.path.join(base_dir, 'Games_dataset')
steam_data = os.path.join(data_dir, 'steam_games.csv')
game_data_old = pd.read_csv(steam_data)

In [21]:
game_data_old

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

In [28]:
def get_attr_set(column_name):
    res = set()
    for row in game_data_old[column_name]:
        if type(row) is str:
            res = res.union(set(map(lambda x: x.lower(), row.split(','))))
    return res

In [29]:
tags_set = get_attr_set('popular_tags')
num_players_set = get_attr_set('game_details')

# 80k games dataset
- https://www.kaggle.com/deepann/80000-steam-games-dataset
- Contains game info up to Dec 2020


In [33]:
dir_80k = os.path.join(data_dir, '80k_games')
game_data_80k = pd.read_csv(os.path.join(dir_80k, 'steam_data.csv'))
text_data_80k = pd.read_csv(os.path.join(dir_80k, 'text_content.csv'))

In [34]:
game_data_80k

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...
...,...,...,...,...,...,...,...,...,...,...,...,...
81043,https://store.steampowered.com/bundle/2961/Ste...,-,-,-,-,-,-,-,-,Buy Steam: Rails to Riches - All Map Expansion...,-,-
81044,https://store.steampowered.com/bundle/3123/Det...,-,-,-,-,-,-,-,-,Buy Detective Hank and the Golden Sneeze - Gam...,-,-
81045,https://store.steampowered.com/bundle/3175/Fea...,-,-,-,-,-,-,-,-,-,-,-
81046,https://store.steampowered.com/bundle/3176/Fea...,-,-,-,-,-,-,-,-,-,-,-


In [35]:
text_data_80k

,url,desc,full_desc,requirements,popu_tags
0,https://store.steampowered.com/app/945360/Amon...,An online and local party game of teamwork and...,About This GamePlay with 4-10 player online or...,System RequirementsMinimum:OS: Windows 7 SP1+P...,MultiplayerOnline Co-OpSpaceSocial DeductionSu...
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive (CS: GO) expa...,About This GameCounter-Strike: Global Offensiv...,System RequirementsWindowsMac OS XSteamOS + Li...,FPSShooterMultiplayerCompetitiveActionTeam-Bas...
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys is a massively multiplayer party gam...,About This GameFall Guys: Ultimate Knockout fl...,System RequirementsMinimum:Requires a 64-bit p...,MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
3,https://store.steampowered.com/app/1158310/Cru...,Paradox Development Studio brings you the sequ...,About This GameParadox Development Studio brin...,System RequirementsWindowsMac OS XSteamOS + Li...,StrategyRPGSimulationMedievalGrand StrategyHis...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2 is an action MMO with a single evolv...,About This GameDive into the world of Destiny ...,System RequirementsMinimum:Requires a 64-bit p...,Free to PlayLooter ShooterMultiplayerFPSAction...
...,...,...,...,...,...
81043,https://store.steampowered.com/bundle/2961/Ste...,-,-,-,-
81044,https://store.steampowered.com/bundle/3123/Det...,-,About this bundleThis bundle contains the base...,-,-
81045,https://store.steampowered.com/bundle/3175/Fea...,-,-,-,-
81046,https://store.steampowered.com/bundle/3176/Fea...,-,-,-,-


In [216]:
def parse_url(url):
    lst = [m.start() for m in re.finditer(r"/", url)]
    uniq_id = 0
    is_app = False
    if len(lst) > 4:
        uniq_id = int(url[lst[3]+1 : lst[4]])
        is_app = url[lst[2]+1 : lst[3]] == 'app'
    return uniq_id, is_app
    
def clean_name(name):
    name = re.findall("[ A-Za-z0-9_'./#&-:]", name)
    name_cleaned = ''.join(name).strip()
    name_simplified = re.split("[0-9&:]", name_cleaned)[0].strip()
    return name_cleaned, name_simplified
    
def parse_tags(tag_str, existing_tags_set):
    tag_set = set()
    tag_str = re.sub(r'\s+', '', tag_str).lower()
    for tag in existing_tags_set:
        tag_concat = re.sub(r'\s+', '', tag).lower()
        if tag_concat in tag_str:
            tag_set.add(tag)
    return list(tag_set)

def generate_genre(tags_lst, inv_genre_dict):
    genre_set = set()
    for tag in tags_lst:
        genre_set = genre_set.union(inv_genre_dict.get(tag, set()))
        for word in tag.split():
            genre_set = genre_set.union(inv_genre_dict.get(word, set()))
    return list(genre_set)

def parse_num_players(tags_lst):
    num_players_set = set()
    for tag in tags_lst:
        if tag == 'singleplayer':
            num_players_set.add('single-player')
        if tag == 'online multiplayer' or 'co-op' in tag.split() or 'multiplayer' in tag.split():
            num_players_set.add('multi-player')
        else:
            num_players_set.add('single-player')
    return list(num_players_set)

def get_rating(rating):
    try:
        idx = rating.index('%')
        bracket_start, bracket_end = rating.index('('), rating.index(')')
        score = int(rating[idx - 2 : idx])
        num_reviews = int(''.join(re.findall("[0-9]+", rating[bracket_start: bracket_end])))
        return score, num_reviews
    except ValueError:
        return 0, 0
    
def get_attr(uniq_id, column, column_inv_index):
    res = set()
    if type(column) is str:
        res = res.union(set(map(lambda x: x.lower().strip(), column.split(','))))
        for r in res:
            column_inv_index.setdefault(r, []).append(uniq_id)
    return list(res), column_inv_index

In [210]:
existing_tags_set = tags_set.union(set(['social deduction', 'psychological', 'rogue like', 'rogue lite', 
                               'party game']))

genre_list_wiki = ['action', 'adventure', 'role-playing', 'simulation', 'strategy', 'sports', 
                   'mmo', 'open-world', 'boardgame', 'indie', 'party-game']

genre_dict = dict()
genre_dict['action'] = ['action', 'platform', 'shooter', 'fps', 'fighter', 'fighting', 'beat \'em up', 
                        'stealth', 'survival', 'battle royale', 'rhythm', 'hack and slash', 'psychological']
genre_dict['adventure'] = ['adventure', 'text-based', 'visual novel', 'movie', '3d vision']
genre_dict['role-playing'] = ['role playing', 'rpg', 'mmorpg', 'rogue', 'rogue-like', 'rogue-lite', 'jrpg']
genre_dict['simulation'] = ['simulation', 'management']
genre_dict['strategy'] = ['strategy', '4x', 'moba', 'rts', 'real-time', 'rtt', 'tower defense', 
                          'turn-based', 'wargame', 'board game', 'logic']
genre_dict['sports'] = ['sports', 'e-sports', 'racing']
genre_dict['mmo'] = ['mmo', 'mmorpg', 'massively multiplayer', 'online']
genre_dict['open-world'] = ['open word', 'sandbox']
genre_dict['boardgame'] = ['board game']
genre_dict['indie'] = ['indie']
genre_dict['party-game'] = ['party', 'party-based']

inverse_genre_dict = dict()
for (genre, keyword_lst) in genre_dict.items():
    for word in keyword_lst:
        inverse_genre_dict.setdefault(word, set()).add(genre)

In [217]:
inv_genre = dict()
inv_tag = dict()
inv_developer = dict()
inv_publisher = dict()
inv_name_simp = dict()
inv_num_players = dict()
id_to_info = dict()

total_games = len(game_data_80k)
for i in range(total_games):
    if type(game_data_80k['url'][i]) is not str or type(game_data_80k['name'][i]) is not str:
        continue
        
    uniq_id, is_app = parse_url(game_data_80k['url'][i])
    if uniq_id == 0 or uniq_id in id_to_info:
        continue
        
    name, simp_name = clean_name(game_data_80k['name'][i])
    inv_name_simp.setdefault(simp_name, []).append(uniq_id)
    
    developer, inv_developer = get_attr(uniq_id, game_data_80k['developer'][i], inv_developer)
    publisher, inv_publisher = get_attr(uniq_id, game_data_80k['publisher'][i], inv_publisher)
    
    assert (parse_url(text_data_80k['url'][i])[0] == uniq_id)
    tags_lst = parse_tags(text_data_80k['popu_tags'][i], existing_tags_set)
    for tag in tags_lst:
        inv_tag.setdefault(tag, []).append(uniq_id)
    genre_lst = generate_genre(tags_lst, inverse_genre_dict)
    for genre in genre_lst:
        inv_genre.setdefault(genre, []).append(uniq_id)
    num_players = parse_num_players(tags_lst)
    for player in num_players:
        inv_num_players.setdefault(player, []).append(uniq_id)
        
    rating = get_rating(game_data_80k['user_reviews'][i])[0]
    short_bio = text_data_80k['desc'][i]
    full_bio = text_data_80k['full_desc'][i]
    if 'About This Game' in full_bio:
        full_bio = full_bio[15:]
        
    pegi = game_data_80k['pegi'][i]
    
    mature = False
    if type(pegi) == str and re.findall("[a-z0-9]", pegi):
        mature = True
    if 'sexual content' in tags_lst or 'nudity' in tags_lst:
        mature = True
        
    info = {'name': name, 'simplified_name': simp_name, 'developer': developer, 'publisher': publisher, 
           'tags': tags_lst, 'genre': genre_lst, 'number_of_players': num_players, 'short_bio': short_bio, 
            'full_bio': full_bio, 'rating': rating, 'mature_content': mature, 'is_app': is_app}
    id_to_info[uniq_id] = info      

In [221]:
len(inv_tag['colorful'])

2671

In [ ]:
len(inv_tag['colorful'])

In [220]:
inv_name_simp

{'Among Us': [945360],
 'Counter-Strike': [730, 240, 10, 80],
 'Fall Guys': [1097150, 1343612],
 'Crusader Kings III': [1158310, 1303182, 1303184, 1303183, 1296731],
 'Destiny': [1085660, 1090150, 1330040, 1314563, 1090200, 1314561, 1314562],
 'Hades': [1145360],
 'Sea of Thieves': [1172620],
 "Tom Clancy's Rainbow Six Siege": [359550],
 'Dota': [570],
 'Warframe': [230410, 1088780, 1390110, 1390131, 1390132, 1390130],
 'The Elder Scrolls Online': [306130, 1382241],
 'Path of Exile': [238960],
 'Borderlands': [397540,
  49520,
  261640,
  991260,
  1232256,
  224180,
  1233060,
  213230,
  230090,
  218550,
  213210,
  872280,
  207870,
  1232257,
  213250,
  1233061,
  245930,
  213780,
  208690,
  245970,
  245950,
  224200,
  245890,
  245910,
  225833,
  225830,
  225832,
  225831,
  224143,
  225834,
  224166,
  225826,
  224164,
  225821,
  224161,
  225827,
  224145,
  225824,
  225829,
  225823,
  224163,
  224165,
  224168,
  224142,
  225820,
  224160,
  225825,
  224144,
  2

In [213]:
id_to_info

{945360: {'name': 'Among Us',
  'simplified_name': 'Among Us',
  'developer': ['innersloth'],
  'publisher': ['innersloth'],
  'tags': ['colorful',
   'indie',
   'pvp',
   'psychological',
   'survival',
   'minigames',
   'cartoony',
   'logic',
   'local multiplayer',
   'top-down',
   'local co-op',
   'co-op',
   'funny',
   'casual',
   'space',
   'cartoon',
   'aliens',
   'social deduction',
   'multiplayer',
   'online co-op',
   'sci-fi',
   '2d'],
  'genre': ['action', 'indie', 'mmo', 'strategy'],
  'number_of_players': ['multi-player', 'single-player'],
  'short_bio': 'An online and local party game of teamwork and betrayal for 4-10 players...in space!',
  'full_bio': "Play with 4-10 player online or via local WiFi as you attempt to prepare your spaceship for departure, but beware as one or more random players among the Crew are Impostors bent on killing everyone!Originally created as a party game, we recommend playing with friends at a LAN party or online using voice chat

In [214]:
#Write results to json
out_dir = json_dir = os.path.join(os.getcwd(), '80k_data')
with open(os.path.join(out_dir, 'inv_game_tags.json'), 'w') as json_file:
    json.dump(inv_tag, json_file)
with open(os.path.join(out_dir, 'inv_game_genre.json'), 'w') as json_file:
    json.dump(inv_genre, json_file)
with open(os.path.join(out_dir, 'inv_game_developer.json'), 'w') as json_file:
    json.dump(inv_developer, json_file)
with open(os.path.join(out_dir, 'inv_game_publisher.json'), 'w') as json_file:
    json.dump(inv_publisher, json_file)
with open(os.path.join(out_dir, 'inv_game_name_simplified.json'), 'w') as json_file:
    json.dump(inv_name_simp, json_file)
with open(os.path.join(out_dir, 'inv_game_num_players.json'), 'w') as json_file:
    json.dump(inv_num_players, json_file)
with open(os.path.join(out_dir, 'id_to_info.json'), 'w') as json_file:
    json.dump(id_to_info, json_file)

In [ ]:
#zip and delete id_to_info.json
